In [133]:
# importer les librairies
import pandas as pd 


# nlp processing & corpus
from stop_words import get_stop_words
import nltk
from nltk.probability import FreqDist


## import punkt algo (spliting text into tokens)
nltk.download('punkt')

# Stylecloud viz
import stylecloud


[nltk_data] Downloading package punkt to /Users/richard/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [134]:
# import datas

# local path from 
url = 'SMSSpamCollection.txt'

# Read the text file into a DataFrame
df = pd.read_csv(url, delimiter='\t', header=None, names=['label', 'message'])

# Print the first few rows of the DataFrame
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


*Label Ham & Spam*


In [135]:
# how do they split ? 
count_val = df['label'].value_counts(normalize=True).to_frame().reset_index().rename(columns={'index':'label','label':'count_pct'})
count_val
# most of Ham 

,label,count_pct
0,ham,0.865937
1,spam,0.134063


In [136]:
# label = ham 
df_ham = df.copy()
df_ham = df_ham.loc[df_ham['label'] =='ham']

# label = spam
df_spam = df.copy()
df_spam = df_spam.loc[df_spam['label'] =='spam']

## SPAM Messages ## 

In [137]:
len(df_spam['message'].unique()) / len(df_spam['message'])
# 87 % unique values 

0.8741633199464525

In [138]:
# Spam message present more than once are : 
spam_val_count = df_spam['message'].value_counts().to_frame().reset_index().rename(columns={'index':'msg','message':'count'})

In [139]:
# present more than once, maybe supposed better to scam peole ? 
most_spam = spam_val_count.loc[spam_val_count['count']>1]

### most present words on message ### 

In [140]:
# define a list of stop_words from the english most common
# list  
stop_words = get_stop_words('english')

In [141]:
# custom the stop_words list with 

# my_new_stop_word = []

#stop_words.append('my_new_stop_word')
#stop_words.remove('my_new_stop_word')

In [142]:
# Tokenize the msg
most_spam['tokens'] = most_spam['msg'].apply(nltk.word_tokenize)

/var/folders/mb/fck_zr_56_n2ss6_66qj2cn40000gn/T/ipykernel_5200/1467994210.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Example for this token selection #### 

In [143]:
# Remove stop words
most_spam['tokens'] = most_spam['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

# Remove non-alphabetic words
most_spam['tokens'] = most_spam['tokens'].apply(lambda x: [word for word in x if word.isalpha()])

# Compute word frequency
words = []
for msg in most_spam['tokens']:
    for word in msg:
        words.append(word)

freq_dist = FreqDist(words)

# Print the most common words
freq_dist.most_common(10)

# Convert FreqDist to DataFrame
freq_word = pd.DataFrame(list(freq_dist.items()), columns=['word', 'count'])

/var/folders/mb/fck_zr_56_n2ss6_66qj2cn40000gn/T/ipykernel_5200/1613764838.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/mb/fck_zr_56_n2ss6_66qj2cn40000gn/T/ipykernel_5200/1613764838.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [144]:
# Convert FreqDist to DataFrame
freq_word = pd.DataFrame(list(freq_dist.items()), columns=['word', 'count'])

# sort_values & top 7 / 608 (1%)
freq_word.sort_values(by='count',ascending=False).head(7)

,word,count
14,now,21
68,Call,19
47,ur,19
1,call,19
43,FREE,16
51,txt,14
118,mobile,14


### Stylecloud ###

In [145]:
# Create a dictionary of words and their counts
word_freq = dict(zip(freq_word['word'], freq_word['count']))

# Generate a word cloud twitter output
stylecloud.gen_stylecloud(text=word_freq, icon_name='fab fa-twitter', output_name='twitter.png')


In [146]:
# Generate a word cloud phone output
stylecloud.gen_stylecloud(text=word_freq, icon_name='fas fa-mobile-alt', output_name='mobile.png')

### function definition ### 

In [160]:
# def de la function most_common()
## input 
## - dataset       = un dataset
## - col_of_tokens = col of the dataset (whith tokens of the orignal messages)
#
#
# Filtering while processing :  
## Stop words + non alphabetic words - OFF 
#
### RETURN dataset with word / count 
#
def most_common(dataset,col_of_tokens):
    dataset = dataset
    col_of_tokens = str(col_of_tokens)

    # Remove stop words
    dataset[col_of_tokens] = most_spam[col_of_tokens].apply(lambda x: [word for word in x if word.lower() not in stop_words])

    # Remove non-alphabetic words
    dataset[col_of_tokens] = dataset[col_of_tokens].apply(lambda x: [word for word in x if word.isalpha()])

    # Compute word frequency
    words = []
    for msg in dataset['tokens']:
        for word in msg:
            words.append(str(word))

    freq_dist = FreqDist(words)

    # Print the most common words
    # freq_dist.most_common(10)

    # Convert FreqDist to DataFrame
    freq_word = pd.DataFrame(list(freq_dist.items()), columns=['word', 'count'])

    # return du dataset
    return freq_word

In [148]:
# apply de function 
# my_df = most_common(most_spam,'tokens')

### other spams ### 

In [161]:
spam_val_count['tokens'] = spam_val_count['msg'].apply(nltk.word_tokenize)

In [166]:
spam_val_count.iloc[648]['msg']

'You have WON a guaranteed £1000 cash or a £2000 prize.To claim yr prize call our customer service representative on'

In [ ]:
s

In [ ]:
# Remove stop words
spam_val_count['tokens'] = spam_val_count['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

# Remove non-alphabetic words
spam_val_count['tokens'] = spam_val_count['tokens'].apply(lambda x: [word for word in x if word.isalpha()])

# Compute word frequency
words = []
for msg in spam_val_count['tokens']:
    for word in msg:
        words.append(word)

freq_dist = FreqDist(words)

# Print the most common words
freq_dist.most_common(10)

# Convert FreqDist to DataFrame
freq_word = pd.DataFrame(list(freq_dist.items()), columns=['word', 'count'])